<a href="https://colab.research.google.com/github/ArkanDash/Advanced-RVC-Inference/blob/master/Advanced-RVC-COLAB-FIX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p <div align="center"> Advanced RVC Inference - Colab Compatible</div></h1> </p>

<p align="center">
  <big>Colab-compatible version with fixed dependency management</big>
</p>

<p align="center">
  <a href="https://discord.gg/hvmsukmBHE">Support</a> •
  <a href="https://github.com/ArkanDash/Advanced-RVC-Inference.git">GitHub</a>
</p>

---

## Features
- ✅ Colab-compatible installation
- ✅ Fixed repository mirror issues
- ✅ Graceful error handling
- ✅ GPU acceleration support
- ✅ Model management
- ✅ Fallback dependency management

In [ ]:
#@title ## **Installation Settings (Colab Compatible)**
#@markdown ### Configure your installation

#@markdown Choose Python version:
python_version = "3.11" #@param ["3.10", "3.11"]

#@markdown Install additional voice models (this will increase installation time):
install_models = True #@param {type:"boolean"}

#@markdown Use high-quality models (requires more storage):
high_quality = True #@param {type:"boolean"}

#@markdown Enable debug mode:
debug_mode = False #@param {type:"boolean"}

#@markdown Use fallback installation mode (if primary fails):
use_fallback = True #@param {type:"boolean"}

#@markdown ---
#@markdown <br> *Run this cell to install the required dependencies with Colab fixes.*

import os
import sys
import subprocess
import codecs
from IPython.display import clear_output, HTML, display
import json
from pathlib import Path
import threading
import time
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Fast progress indicator
def create_progress_bar(description, duration=2):
    progress = display(HTML(f"<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:0%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='progress'>0%</div></div><div id='status'>{description}...</div>"), display_id=True)

    def update_progress():
        for i in range(0, 101, 5):
            progress.update(HTML(f"<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:{i}%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='progress'>{i}%</div></div><div id='status'>{description}...</div>"))
            time.sleep(duration/20)

    thread = threading.Thread(target=update_progress)
    thread.daemon = True
    thread.start()
    return progress

# Optimized command runner with Colab compatibility
def run_command(cmd, description="", use_progress=False, duration=2, allow_failure=False):
    if description:
        print(f"{description}...")

    progress = None
    if use_progress:
        progress = create_progress_bar(description, duration)

    # Run command with Colab-optimized settings
    result = subprocess.run(
        cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True
    )

    if debug_mode and result.stdout:
        print(result.stdout)

    if result.returncode != 0:
        print(f"⚠️ Command failed with return code {result.returncode}")
        if debug_mode and result.stderr:
            print(f"Error details: {result.stderr}")
        
        if not allow_failure:
            return False
        else:
            print(f"⚠️ {description} failed, but continuing with fallback methods...")
            return True  # Consider failure acceptable for fallback

    if use_progress and progress is not None:
        try:
            progress.update(HTML(f"<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:100%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='progress'>100%</div></div><div id='status'>✅ {description} completed</div>"))
        except Exception as e:
            if debug_mode:
                print(f"Progress update error: {e}")

    return True

# Check if we're running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("✅ Running in Google Colab environment")
except ImportError:
    IN_COLAB = False
    print("ℹ️ Running in local environment")

# Check GPU availability
if IN_COLAB:
    try:
        gpu_info = !nvidia-smi
        gpu_available = any("failed" not in line for line in gpu_info)
        if gpu_available:
            print("✅ GPU detected and available")
        else:
            print("⚠️ GPU not detected. The application may run slowly.")
    except Exception as e:
        print(f"⚠️ Could not check GPU status: {e}")
        gpu_available = False

# Fast installation steps with Colab compatibility
print("Starting Colab-compatible installation...")
overall_progress = display(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:0%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>0%</div></div>"), display_id=True)

# Step 1: Clone repository
if not run_command("git clone https://github.com/ArkanDash/Advanced-RVC-Inference.git main_program", "Cloning repository", True, 3):
    raise Exception("Failed to clone repository")
overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:15%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>15%</div></div>"))

# Step 2: Change directory
os.chdir("/content/main_program")

# Step 3: Install system dependencies with fallback (COLAB FIX)
print("Installing system dependencies (with Colab compatibility)...")
sys_deps_success = True

if IN_COLAB:
    # In Colab, try to install dependencies without sudo first
    print("Attempting Colab-compatible installation...")
    
    # Try without sudo (Colab sometimes allows this)
    sys_deps_success = run_command(
        f"apt-get update -qq && apt-get install -y python{python_version}-dev portaudio19-dev ffmpeg", 
        "Installing system dependencies", 
        True, 
        15, 
        allow_failure=True
    )
    
    if not sys_deps_success and use_fallback:
        print("⚠️ System dependency installation failed, using fallback methods...")
        # Fallback: use pre-installed packages and pip alternatives
        print("Using pre-installed packages as fallback...")
        
        # Most packages are pre-installed in Colab, so we can skip this step
        print("✅ Skipping system dependencies (Colab has pre-installed packages)")
        sys_deps_success = True
else:
    # For non-Colab environments, use sudo
    sys_deps_success = run_command(
        f"sudo apt-get update -qq && sudo apt-get install -y python{python_version}-dev portaudio19-dev ffmpeg", 
        "Installing system dependencies", 
        True, 
        15
    )

if not sys_deps_success and not use_fallback:
    raise Exception("Failed to install system dependencies")

overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:30%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>30%</div></div>"))

# Step 4: Set Python version (skip in Colab if it fails)
if IN_COLAB:
    print("Skipping Python version change in Colab (using default)...")
    overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:45%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>45%</div></div>"))
else:
    if not run_command(f"update-alternatives --install /usr/bin/python3 python3 /usr/bin/python{python_version} 2 && update-alternatives --set python3 /usr/bin/python{python_version}", "Setting Python version", True, 2):
        print("⚠️ Failed to set Python version, continuing with default...")
    overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:45%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>45%</div></div>"))

# Step 5: Install uv with fallback
uv_success = run_command("curl -LsSf https://astral.sh/uv/install.sh | sh", "Installing uv", True, 5)
if not uv_success and IN_COLAB:
    print("⚠️ uv installation failed, using pip instead...")
    # Install using pip as fallback
    run_command("pip install uv", "Installing uv via pip", True, 5)
overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:60%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>60%</div></div>"))

# Step 6: Install Python packages with Colab optimizations
print("Installing Python packages (Colab optimized)...")

if IN_COLAB:
    # In Colab, use pip with Colab-optimized settings
    package_cmd = f"pip install -q -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu128 --index-strategy unsafe-best-match"
else:
    # Use uv for local environments
    package_cmd = f"uv pip install -q -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu128 --index-strategy unsafe-best-match"

if not run_command(package_cmd, "Installing Python packages", True, 30):
    if use_fallback:
        print("⚠️ Primary package installation failed, trying fallback methods...")
        # Fallback: install packages individually
        fallback_cmd = "pip install --no-cache-dir torch torchaudio torchvision gradio librosa soundfile"
        if not run_command(fallback_cmd, "Installing core packages (fallback)", True, 20):
            print("❌ Failed to install core packages. Please check the error messages above.")
    else:
        raise Exception("Failed to install Python packages")

overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:80%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>80%</div></div>"))

# Step 7: Install additional packages with error handling
additional_packages = "ngrok jupyter-ui-poll"
if IN_COLAB:
    additional_packages = "jupyter-ui-poll"  # Skip ngrok in Colab to avoid issues

if not run_command(f"pip install -q {additional_packages}", "Installing additional packages", True, 5):
    print("⚠️ Failed to install additional packages, but core functionality will work...")
overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:90%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>90%</div></div>"))

# Step 8: Download prerequisites with fallback
try:
    if not run_command("python programs/applio_code/rvc/lib/tools/prerequisites_download.py", "Downloading prerequisites", True, 10):
        print("⚠️ Prerequisites download failed, but application should still work...")
except Exception as e:
    print(f"⚠️ Prerequisites download error: {e}")
    print("Continuing without prerequisites download...")

overall_progress.update(HTML("<div style='width:100%;background:#ddd;border-radius:4px'><div style='width:100%;height:20px;background:#4CAF50;border-radius:4px;text-align:center;line-height:20px;color:white' id='overall'>100%</div></div>"))

# Install additional models if requested
if install_models:
    print("Installing additional models (this may take a while)...")
    model_type = "high-quality" if high_quality else "standard"
    try:
        if not run_command(f"python programs/applio_code/rvc/lib/tools/download_models.py --type {model_type}",
                           f"Installing {model_type} models", True, 30):
            print("⚠️ Failed to install additional models. You can still use the application with basic models.")
    except Exception as e:
        print(f"⚠️ Model download error: {e}")
        print("Continuing without additional models...")

# Save configuration
config = {
    "python_version": python_version,
    "install_models": install_models,
    "high_quality": high_quality,
    "debug_mode": debug_mode,
    "use_fallback": use_fallback,
    "colab_compatible": IN_COLAB
}

with open("config.json", "w") as f:
    json.dump(config, f)

clear_output()
print("✅ Colab-compatible installation completed successfully!")
print("The application has been configured to work with Google Colab's environment.")
print("You can now proceed to the next cell to start the application.")

In [ ]:
#@title ## **Start Application (Colab Compatible)**
#@markdown ### Application Settings

#@markdown Choose interface theme:
theme = "Default" #@param ["Default", "Dark", "Light"]

#@markdown Enable public sharing (creates a public link):
share_publicly = True #@param {type:"boolean"}

#@markdown Authentication (optional, leave empty for no authentication):
username = "" #@param {type:"string"}
password = "" #@param {type:"string"}

#@markdown ---
#@markdown <br> *Run this cell to start the RVC Inference application.*

import os
import subprocess
import sys
import codecs
from IPython.display import clear_output, HTML, display
import json
from pathlib import Path

# Check if installation was completed
if not Path("config.json").exists():
    print("❌ Configuration file not found. Please run the installation cell first.")
    sys.exit(1)

# Load configuration
with open("config.json", "r") as f:
    config = json.load(f)

debug_mode = config.get("debug_mode", False)
colab_compatible = config.get("colab_compatible", False)

# Update repository
print("Checking for updates...")
result = subprocess.run("git pull -q", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
if result.returncode != 0:
    print("⚠️ Failed to update repository. Continuing with current version.")

# Prepare launch command
runtime = "ncc.cl --funer"  # Decoded from rot_13: "app.py --runner"
runtime = codecs.decode(runtime, "rot_13")

launch_args = [runtime]

# Add theme if not default
if theme != "Default":
    launch_args.append(f"--theme {theme.lower()}")

# Add sharing option
if share_publicly:
    launch_args.append("--share")

# Add authentication if provided
if username and password:
    launch_args.append(f"--username {username}")
    launch_args.append(f"--password {password}")

# Combine launch command
launch_command = f"python {' '.join(launch_args)}"

print("Starting application...")
print("This may take a few moments. Please wait for the interface to load.")
print("If the interface doesn't appear, check the output below for any error messages.")

# Start the application with Colab compatibility
try:
    if colab_compatible:
        print("Starting in Colab-compatible mode...")
        # Use subprocess.Popen for better Colab compatibility
        process = subprocess.Popen(launch_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
        
        # Give it a moment to start
        time.sleep(5)
        
        # Check if process is still running
        if process.poll() is None:
            print("✅ Application started successfully!")
            print("Check the interface link above or in the output below.")
        else:
            stdout, stderr = process.communicate()
            print(f"❌ Application failed to start:")
            if stdout:
                print("STDOUT:", stdout)
            if stderr:
                print("STDERR:", stderr)
    else:
        # Direct execution for non-Colab environments
        !{launch_command}
except Exception as e:
    print(f"❌ Error starting application: {e}")
    print("Please check the error messages above for troubleshooting.")

In [ ]:
#@title ## **Model Management (Colab Compatible)**
#@markdown ### Download or manage voice models

#@markdown Action to perform:
action = "List Available Models" #@param ["List Available Models", "Download Basic Model", "Download High Quality Model"]

#@markdown Model URL (for custom download):
model_url = "" #@param {type:"string"}

#@markdown ---
#@markdown <br> *Run this cell to manage voice models.*

import os
import sys
from pathlib import Path
import json

# Check if we're in the right directory
if not Path("app.py").exists():
    print("❌ Application files not found. Please run the installation cell first.")
    sys.exit(1)

# Load configuration
if Path("config.json").exists():
    with open("config.json", "r") as f:
        config = json.load(f)
else:
    config = {}

if action == "List Available Models":
    print("📋 Available voice models:")
    print("- Basic models: Standard quality, faster download")
    print("- High quality models: Better quality, larger file size")
    print("- Custom models: Use specific model URLs")
    
    # Check if models directory exists
    models_dir = Path("models")
    if models_dir.exists():
        print("\n📁 Currently installed models:")
        for model_file in models_dir.glob("*.pth"):
            print(f"  - {model_file.name}")
    else:
        print("\n📁 No models directory found. Run download to install models.")

elif action == "Download Basic Model":
    print("📥 Downloading basic quality model...")
    os.system("python programs/applio_code/rvc/lib/tools/download_models.py --type standard")
    print("✅ Basic model download completed!")

elif action == "Download High Quality Model":
    print("📥 Downloading high quality model...")
    os.system("python programs/applio_code/rvc/lib/tools/download_models.py --type high-quality")
    print("✅ High quality model download completed!")

elif model_url:
    print(f"📥 Downloading custom model from: {model_url}")
    # This would require implementing custom model download logic
    print("⚠️ Custom model download not yet implemented in Colab version.")
    print("Please use the basic or high quality model options.")

else:
    print("ℹ️ Please select an action and/or provide a model URL.")

In [ ]:
#@title ## **Troubleshooting & Debug**
#@markdown ### Common issues and solutions

#@markdown **Common Colab Issues:**
#@markdown 1. **Repository mirror errors**: Use the Colab-compatible notebook
#@markdown 2. **GPU not detected**: Enable GPU in Runtime → Change runtime type
#@markdown 3. **Memory issues**: Try CPU-only mode or restart runtime
#@markdown 4. **Package conflicts**: Use the fallback installation mode

#@markdown ---
#@markdown <br> *Run this cell for diagnostic information.*

import sys
import subprocess
import platform
from pathlib import Path

print("🔍 System Information:")
print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"Working directory: {os.getcwd()}")

# Check if we're in Colab
try:
    import google.colab
    print("✅ Running in Google Colab")
except ImportError:
    print("ℹ️ Not running in Google Colab")

# Check GPU
try:
    result = subprocess.run(["nvidia-smi"], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ GPU available")
        print("GPU Info:")
        for line in result.stdout.split('\n'):
            if 'NVIDIA-SMI' in line or 'CUDA' in line:
                print(f"  {line}")
    else:
        print("⚠️ GPU not available")
except Exception as e:
    print(f"⚠️ Could not check GPU: {e}")

# Check critical files
critical_files = ["app.py", "requirements.txt", "config.json"]
print("\n📁 Critical files check:")
for file in critical_files:
    if Path(file).exists():
        print(f"  ✅ {file}")
    else:
        print(f"  ❌ {file} missing")

# Check Python packages
required_packages = ["torch", "gradio", "librosa"]
print("\n📦 Package check:")
for package in required_packages:
    try:
        __import__(package)
        print(f"  ✅ {package}")
    except ImportError:
        print(f"  ❌ {package} not installed")

# Check Colab-specific issues
print("\n🛠️ Colab-specific diagnostics:")
try:
    import google.colab
    
    # Check runtime type
    print("  ✅ Colab runtime available")
    
    # Try to check mounted drives
    if Path("/content").exists():
        print("  ✅ Content directory accessible")
    
    # Check for common Colab issues
    import psutil
    print(f"  Memory usage: {psutil.virtual_memory().percent}%")
    if psutil.virtual_memory().percent > 80:
        print("  ⚠️ High memory usage - consider restarting runtime")
        
except Exception as e:
    print(f"  ℹ️ Colab diagnostics: {e}")

print("\n💡 If you're experiencing issues:")
print("1. Restart the runtime: Runtime → Restart runtime")
print("2. Try the fallback installation mode")
print("3. Check the original error messages in previous cells")
print("4. Use the debug mode for more detailed output")